In [1]:
import numpy as np;
import pandas as pd;

In [2]:
mu = 0.01

```
                P1
               /. \
     ----------    -------
     |                   |
     10                 10
     |                   |
     P1.1 ---- 10 ----- P1.2           
     / \                / \
    10  10            10  10 
   /      \          /      \
  P1.1.1  P1.1.2  P1.2.1.  P1.2.2 
```

In [3]:
initDataDF = pd.read_csv("../data/tvm_init_data.csv")

distanceData = pd.read_csv("../data/tvm_distance_matrix.csv")

r = len(initDataDF.index)
initDataDF

,name,S,E,I,H,R
0,Amboori,15920,0,20,0,0
1,Anad,31687,0,0,0,0
2,Anchuthengu,17396,0,0,0,0
3,Andoorkonam,30781,0,0,0,0
4,Aruvikkara,33396,0,0,0,0
...,...,...,...,...,...,...
72,Venganoor,35963,0,0,0,0
73,Vettoor,18704,0,0,0,0
74,Vilappil,36212,0,0,0,0
75,Vilavoorkal,31761,0,0,0,0


In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)

distanceMatrix[0][4]

19952

In [5]:
def getSij(i, j, distanceMatrix, initDataDF):
    #print(distanceMatrix[i] <= distanceMatrix[i][j])
    return(initDataDF[distanceMatrix[i] <= distanceMatrix[i][j]]['S'].sum() - initDataDF.iloc[i]['S'] - initDataDF.iloc[j]['S'])
    


In [6]:
getSij(63,23,distanceMatrix, initDataDF)

0

In [7]:
def getTij(i,j, distanceMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    Ti = mu*Ni

    if(i==j):
        return Ni - Ti;
    
    Nj = initDataDF.iloc[j]['S']
    Sij = getSij(i,j,distanceMatrix, initDataDF)
    Tij = Ti * ((Ni * Nj) /((Ni + Sij)*(Ni + Nj + Sij)))
    return Tij
    
    

In [8]:
print(initDataDF.iloc[63]['name'])
print(initDataDF.iloc[23]['name'])

print(getTij(63,23,distanceMatrix, initDataDF))
#print(getTij(26,23,distanceMatrix, initDataDF))


Thiruvananthapuram(C)
Karakulam
497.2141021296551


In [9]:
data = []

#resultsDF = pd.DataFrame(data)

for i in range(0,r-1):
    for j in range(0,r-1):
        Tij = getTij(i,j,distanceMatrix, initDataDF)
        #print("(", i, ",", j, ") ",  initDataDF.iloc[i]['name'], " ", initDataDF.iloc[j]['name'], " " ,Tij)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);


In [11]:
#resultsDF.info
resultsDF.to_csv("../output/tmatrix_results.csv")



In [ ]:
resultsDF[resultsDF['iName']=="Thiruvananthapuram(C)"].head(30)

In [ ]:
resultsDF.loc[4811]